In [1]:
#PLEASE RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import pandas as pd
import numpy as np
import random
import math

In [3]:
#https://pythonforundergradengineers.com/deploy-jupyter-notebook-voila-heroku.html#test-voila-locally
from IPython.display import clear_output

import matplotlib.pyplot as plt
from ipywidgets import interactive, interact, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline

In [4]:
# List of possible outcomes:
# 0 = Command
# 1 = Hit
# 2 = Flee

outcomes = [0,1,0.1]

In [5]:
# Dice all have same # of command except British
def dice(weight_hit = 0.5, weight_command = 1/3):  
    weight_flee = 1 - weight_command - weight_hit
    
    weights = [weight_command, weight_hit, weight_flee]
    return random.choices(outcomes, weights=weights)[0]

# blue or orange dice
def bl_or_dice():
    return dice(weight_hit = 0.5, weight_command = 1/3)

#white or yellow dice
def wh_ye_dice():
    return dice(weight_hit = 1/3, weight_command = 1/3)

# red or purple dice
def re_pu_dice():
    return (dice(weight_hit =0.5, weight_command=0.5))

# green dice
def gr_dice():
    return (dice(weight_hit= 1/3, weight_command=0.5))

In [6]:
##### Number of units to dice conversation #######

# helper functions

# Sets the number of dice to use. Input if a dice is white/yellow or not, to decide max dice # of 2 or 3
def set_num_dice(color_num, white_yellow = False):
    max_dice=2
    if white_yellow:
        max_dice = 3
        
    if color_num > max_dice:
        num_dice = max_dice
    else:
        num_dice = color_num
    
    return num_dice

def throw_one_color(num_dice, color_func):
    results = 0
    for i in range(num_dice):
            results = results+ color_func()
    flees, hits = math.modf(results)
    flees = int(flees*10)
    return flees, hits

In [7]:
#####British and American Roll functions#############
####### Input: number of British or American units (list)
####### Outputs: Total flees by unit color; Total hits by unit color


# Global constants
### because brit/american lists are always organized: "red, yel, orange, green" and "blue, white, purple, green"
yel_whi_lst = [False, True, False, False]
### colored dice functions, aligned to Brit/American color orders in list
brit_color_funcs = [re_pu_dice, wh_ye_dice, bl_or_dice, gr_dice]
ameri_color_funcs = [bl_or_dice, wh_ye_dice, re_pu_dice, gr_dice]

# This function simulates a single roll of multiple dice for one side
# unit_lst = list of # of units per color
def single_roll(unit_lst, america=True):
    num_dice = list(map(set_num_dice, unit_lst, yel_whi_lst))
#    print(num_dice)
    
    if america:
        color_funcs = ameri_color_funcs
    else:
        color_funcs = brit_color_funcs
    results = list(map(throw_one_color, num_dice, color_funcs))
    
    flees, hits = zip(*results)
    
    # We don't care who can "command move" vs. hit, so we just sum hits
    return np.array(flees), int(sum(hits))

In [8]:
####################Calculate which unit to lose during a hit ###########################

# Assuming the order of the lists are [blue, white, purple, green] and [red, yellow, orange, green]
# For Americans, purple = best, so purple = 3; blue = 2nd best, so blue = 2; green = 3rd best, so green = 1
ameri_unit_value = [2,0,3,1]
# For Brits, red = best = 3; purple = 2nd best = 2; green = 3rd best = 1; yellow = worst = 0
brit_unit_value = [3,0,2,1]

# Helper function to find index of least valuable color 
def select_idx(unit_value, subset_indices):
    values_lst = [unit_value[i] for i in subset_indices]
    idx = values_lst.index(min(values_lst))
    return subset_indices[idx]
    

def best_unit_to_lose(unit_lst, prehit_dice, unit_value):
    unit_arr = np.array(unit_lst)
    
    # array of zeroes. Change one value to 1, to select the unit to lose in main "single exchange" function
    best_unit_to_lose = np.array([0,0,0,0])
    
    unit_to_dice_difference = unit_arr - prehit_dice
    # execute "if" when any color has more units than number of dice
    sum_diffs = sum(unit_to_dice_difference)
    if sum_diffs >0:
        max_diff = max(unit_to_dice_difference)
        max_indices = [i for i, j in enumerate(unit_to_dice_difference) if j == max_diff]
        
        # If only one max color, then return that color
        if len(max_indices) == 1:
            best_unit_to_lose[max_indices[0]] = 1
#            print("One Max")
            return best_unit_to_lose
        
        # if multiple, select the worst valued color to hit
        else:
                            ### Helper function
            selected_idx =  select_idx(unit_value, max_indices)
            best_unit_to_lose[selected_idx] = 1
#            print("Multi-Max")
            return best_unit_to_lose
    
    #If losing any color unit will decrease number of dice, then do the below "elif"
    elif sum_diffs == 0:
        # select colors with non-zero number of dice only
        # https://www.kite.com/python/answers/how-to-find-the-index-of-list-elements-that-meet-a-condition-in-python
        colors_to_consider = [idx for idx, element in enumerate(unit_arr) if element > 0]
#        print(colors_to_consider)
        
        # use helper function
        if colors_to_consider != []:
            selected_idx = select_idx(unit_value, colors_to_consider)
            best_unit_to_lose[selected_idx] = 1
#            print("Must lose Dice")
            return best_unit_to_lose      
        else:
            print("Error: No units in battle")
            print(unit_arr)
            print(prehit_dice)
        
    else:
        print("Error")


##########Logic for the above function:############
#If current list of # of units [a,b,c,d] has any element = list of number of dice BEFORE 
#receiving hits [w,x,y,z], then a hit will drop the number of dice for that element/color

#So first: select unit with greatest difference between # of current units and # of dice that could have been rolled. 
#If tie, go to Step two (select element with lowest hit then lowest probability)

#Step two: if all elements with non-zero number of dice could lose a dice next turn, pick the element with the lowest hit.
#If same hit probabilities, select lowest flee.
## so red > orange > green > yellow
## purple > blue > green > white; THOUGH last blue > last purple

In [9]:
####################Functions below to calculate a single exchange of blows####################

# opponent_hits = number of hits your opponent rolled just before
# player_remain = number of units you have remaining before the hits
# player_unit_value = the list of unit values for Brits/Americans
def find_remaining_forces(opponent_hits, player_remain, player_unit_value):
    lost_unit = [0,0,0,0]
    
    for i in range(opponent_hits):
    ## Check that there are any of your units left to hit
        if sum(player_remain)>0:
            ## Find # of dice rolls pre-hit
            prehit_dice = list(map(set_num_dice, player_remain, yel_whi_lst))
            lost_unit = best_unit_to_lose(player_remain, prehit_dice, player_unit_value)
                
#    if sum(lost_unit)==0:
#        print("No units hit")
    
    return player_remain - lost_unit



def single_exchange(brit_unit_lst, ameri_unit_lst, america_defends=True):
    # Remaining units after a dice roll
    brit_remain = np.array(brit_unit_lst[:])
    ameri_remain = np.array(ameri_unit_lst[:])
    
    if america_defends:
        # Check that there is even opponent units to roll against:
        if sum(brit_remain) >0:
            am_flees, am_hits =single_roll(ameri_remain, america=True)
#            print("am hits:{}".format(am_hits))
        
            # calculate remaining American forces
            ameri_remain = ameri_remain - am_flees
        
            # calculate remaining British forces after hit
            brit_remain = find_remaining_forces(am_hits, brit_remain, brit_unit_value)
 #       else:
 #           print("Error1")
        
        
        # Check if British forces survived and your own did not flee
        if (sum(brit_remain)>0) & (sum(ameri_remain)>0):
        # Now remaining British forces attack
            brit_flees, brit_hits =single_roll(brit_remain, america=False)
            
#            print(brit_hits)
#            print(brit_flees)
            
            
            # calculate remaining British forces
            brit_remain = brit_remain - brit_flees
        
            # calculate remaining American forces after hit
            ameri_remain = find_remaining_forces(brit_hits, ameri_remain, ameri_unit_value)
#        else:
#            print("Error2")
#        print("odd loop")
        return brit_remain, ameri_remain
    
    else:
        #Check if there are even opponents to roll against:
        if sum(ameri_remain) >0:
            # British forces attack
            brit_flees, brit_hits =single_roll(brit_remain, america=False)
            
            # calculate remaining British forces
            brit_remain = brit_remain - brit_flees
#            print(brit_hits)
            # calculate remaining American forces after hit
            ameri_remain = find_remaining_forces(brit_hits, ameri_remain, ameri_unit_value);
        
        # Check if American forces survived and your own did not flee
        if (sum(brit_remain)>0) & (sum(ameri_remain)>0):
            
            # America now launches attack
            am_flees, am_hits =single_roll(ameri_remain, america=True)
        
            # calculate remaining American forces
            ameri_remain = ameri_remain - am_flees
        
            # calculate remaining British forces after hit
            brit_remain = find_remaining_forces(am_hits, brit_remain, brit_unit_value);
        
        return brit_remain, ameri_remain

In [10]:
#test_brit = [0,4,0,0]
#test_am = [0,0,4,0]

#single_exchange(test_brit, test_am)

In [11]:
####### Full battle function: Run through all rounds until 1 opponent has zero units ######################

def full_battle(brit_unit_lst, ameri_unit_lst, america_defends=True):
    remaining_brit = np.array(brit_unit_lst)
    remaining_ameri = np.array(ameri_unit_lst)
    
    while (sum(remaining_brit) >0) & (sum(remaining_ameri) > 0):
        remaining_brit, remaining_ameri = single_exchange(remaining_brit, 
                                                          remaining_ameri, 
                                                          america_defends=america_defends)
#        print(remaining_brit, remaining_ameri)

    if sum(remaining_brit) > sum(remaining_ameri):
#        print(remaining_brit)
        return 'red', remaining_brit
    elif sum(remaining_ameri) > sum(remaining_brit):
        return 'blue', remaining_ameri
    elif (sum(remaining_ameri)) ==0 & (sum(remaining_brit)==0):
        return 'error', [0,0,0,0]
    else:
        print("Error!!")

# Simulate battle outcomes for Rebellion:1775

## Key Assumptions:
* No units command away from battle
* When hit, get rid of the color with the most units, UNLESS it decreases your # of dice next throw
* Otheriwse, when all colors can lose a die due to a hit, get rid of the color with the lowest "hit" value

British: Red > Orange > Green  > Yellow

American: Purple > Blue > Green > White

In [12]:
def plot_results(html1,
                 red,yellow,orange,british_green, 
                 html2,
                 blue,white,purple,american_green, 
                 america_defends):
    
    #Take widget inputs and turn into array
    brit_units = np.array([red,yellow,orange,british_green])
    am_units = np.array([blue,white,purple,american_green])
    
    
    winners = []
    remainers = []
    for i in range(1000):
        color, remaining = full_battle(brit_units, am_units, america_defends=america_defends)
        winners.append(color)
        remainers.append(sum(remaining))

    sim_battle_results = pd.DataFrame({"winners":winners, "remaining":remainers})
    total_wins = sim_battle_results['winners'].value_counts()

    victor = total_wins.index[0].replace("'", "")
    print("Winner: {}".format(victor))
    percent_wins = total_wins[0]/len(sim_battle_results)*100
    print("Percent of wins: {}%".format(percent_wins))

    plt.hist(sim_battle_results[sim_battle_results['winners']==victor]['remaining'])
    plt.title("Simulated total remaining units for winnng color")
    plt.xlabel("Remaining units")
    plt.ylabel("# of simulations")
    plt.show()

In [13]:

b = interact(plot_results, 
         html1 = widgets.HTML(value="<b>British Units</b>", placeholder=' ', description=' '),
         
         america_defends = widgets.Checkbox(value=False, description='Select if America is defending'),
         red = widgets.BoundedIntText(value=0,min=0,max=20,step=1, description='# Reds'),
         yellow = widgets.BoundedIntText(value=0,min=0,max=20,step=1, description='# Yellows'),
         orange = widgets.BoundedIntText(value=0,min=0,max=20,step=1, description='# Oranges'),
         british_green = widgets.BoundedIntText(value=0,min=0,max=20,step=1, description='# Br Greens'),
         
         html2 = widgets.HTML(value="<b>American Units</b>", placeholder=' ', description=' '),
         
         blue = widgets.BoundedIntText(value=0,min=0,max=20,step=1, description='# Blues'),
         white = widgets.BoundedIntText(value=0,min=0,max=20,step=1, description='# Whites'),
         purple = widgets.BoundedIntText(value=0,min=0,max=20,step=1, description='# Purples'),
         american_green = widgets.BoundedIntText(value=0,min=0,max=20,step=1, description='# Am Greens')         
     )

interactive(children=(HTML(value='<b>British Units</b>', description=' ', placeholder=' '), BoundedIntText(val…